In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

In [3]:
filename = "../datasets/wonderland.txt"
raw_text = open(filename,"r",encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144583
Total Vocab:  50


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)

Total Patterns:  144483


In [9]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' )

In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss' , verbose=1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs = 20, batch_size = 128, callbacks=callbacks_list)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1129/1129 [==============================] - ETA: 0s - loss: 3.0207
Epoch 1: loss improved from inf to 3.02070, saving model to weights-improvement-01-3.0207.hdf5
1129/1129 [==============================] - 214s 188ms/step - loss: 3.0207
Epoch 2/20
1129/1129 [==============================] - ETA: 0s - loss: 2.8134
Epoch 2: loss improved from 3.02070 to 2.81344, s

In [14]:
# load the network weights
filename = "weights-improvement-20-1.9751.hdf5"

model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [15]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [23]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("Seed:")
print("\"".join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    
print("\nDone.")

Seed:
e"r","
"y"o"u" "k"n"o"w"."”"
"
"a"l"i"c"e" "h"a"d" "b"e"e"n" "l"o"o"k"i"n"g" "o"v"e"r" "h"i"s" "s"h"o"u"l"d"e"r" "w"i"t"h" "s"o"m"e" "c"u"r"i"o"s"i"t"y"." "“"w"h"a"t" "a"
"f"u"n"n"y" "w"a"t"c"h"!"”"  "

“ho ay all ”hu an _nd i can ”ou ” said the mock turtle in a lonat oone tone, “io was an all to tee she wait wo be io!”

“ho woat at all ”arl to t aadad a gad aeiin, in aelned ii the some of the soids so tee  the dould nete the was oo the was of the tood wfth the court, 
“ie you dnn’t know what io ”hu an i connd _ tirn iir ”oul sound ”ou cinne!” the manch hare sniepeyg thete the was soine to tay the wante tf tee soohe the was oo the taateng and toeneng to tee it tar to ced aod toeneng to taek the was oo the thoh the was oo the taaten 
the was oo toee an tole tioe the was oo toe tint hir haad to tee whut hnr tored ana toeken the was oo toe toone. 
“whet _il _aout a lenelen,” said the caterpillar.

“ie you taen to dl ” said the mock turtle in a lonat oone tone, “io was an all to tee 